<a href="https://colab.research.google.com/github/elichen/aoc2024/blob/main/Day_20_Race_Condition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
input = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""

In [41]:
input = open("input.txt").read().rstrip()

In [42]:
from collections import defaultdict, deque
from typing import List, Dict, Set, Tuple

def solve_maze(maze_str: str) -> Dict[int, int]:
    # Convert string maze to 2D grid
    grid = maze_str.split('\n')
    rows, cols = len(grid), len(grid[0])

    # Find start and end positions
    start = end = None
    for i in range(rows):
        for j in range(cols):
            if grid[i][j] == 'S':
                start = (i, j)
            elif grid[i][j] == 'E':
                end = (i, j)

    # First BFS: Find no-skip shortest path and track all positions with their steps
    queue = deque([(start[0], start[1], 0, {start})])
    no_skip_paths = {}  # (row, col) -> steps taken
    shortest_normal = float('inf')

    # Find all positions reachable without skips and their steps
    while queue:
        row, col, steps, path = queue.popleft()

        # Found end
        if grid[row][col] == 'E':
            shortest_normal = min(shortest_normal, steps)
            continue

        # Record position and steps
        pos = (row, col)
        if pos not in no_skip_paths or steps < no_skip_paths[pos]:
            no_skip_paths[pos] = steps

        # Try all directions
        for dx, dy in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
            new_row, new_col = row + dx, col + dy

            # Check bounds and walls
            if not (0 <= new_row < rows and 0 <= new_col < cols):
                continue

            new_pos = (new_row, new_col)
            if new_pos in path:
                continue

            # Only normal moves (no skips)
            if grid[new_row][new_col] == '.' or grid[new_row][new_col] == 'E':
                new_path = path | {new_pos}
                queue.append((new_row, new_col, steps + 1, new_path))

    # Second BFS: Find paths with skips that intersect with no-skip paths
    queue = deque([(start[0], start[1], 0, True, {start})])
    steps_saved = defaultdict(int)

    while queue:
        row, col, steps, can_skip, path = queue.popleft()
        pos = (row, col)

        # If we hit a position in no-skip paths, calculate steps saved
        if not can_skip and pos in no_skip_paths:
            saved = no_skip_paths[pos] - steps
            if saved > 0:
                steps_saved[saved] += 1
            continue

        # Try all directions
        for dx, dy in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
            new_row, new_col = row + dx, col + dy

            # Check bounds
            if not (0 <= new_row < rows and 0 <= new_col < cols):
                continue

            new_pos = (new_row, new_col)
            if new_pos in path:
                continue

            # Normal move through path
            if grid[new_row][new_col] == '.' or grid[new_row][new_col] == 'E':
                new_path = path | {new_pos}
                queue.append((new_row, new_col, steps + 1, can_skip, new_path))

            # Skip through wall if allowed
            elif grid[new_row][new_col] == '#' and can_skip:
                skip_row, skip_col = new_row + dx, new_col + dy
                if (0 <= skip_row < rows and 0 <= skip_col < cols and
                    (grid[skip_row][skip_col] == '.' or grid[skip_row][skip_col] == 'E')):
                    skip_pos = (skip_row, skip_col)
                    if skip_pos not in path:
                        new_path = path | {new_pos, skip_pos}
                        queue.append((skip_row, skip_col, steps + 2, False, new_path))

    return dict(steps_saved)

steps_saved = solve_maze(input)
sum(count for steps, count in steps_saved.items() if steps >= 100)

1323